Selection of SDSS-QSO in XXL-North field
==========================================

We use the catalogue of QSO from SDSS-DR16 to select two samples of sources within the XXL-North field: a set of optical QSO with no X-ray counterparts, and a set of QSO with counterparts in the 4XMM catalogue.

In [ ]:
from pathlib import Path
from mocpy import MOC

data_folder = Path("./data")
xxl_moc_file = data_folder.joinpath("xxl.moc")
xxl_moc = MOC.from_fits(xxl_moc_file)

xxl_sdssqso = xxl_moc.query_vizier_table("VII/289/dr16q", max_rows=None)
xxl_4xmm = xxl_moc.query_vizier_table("IX/59/xmm4dr9s", max_rows=None)

print(f"SDSS-QSO in XXL-North: {len(xxl_sdssqso)} sources")
print(f"4XMM sources in XXL-North: {len(xxl_4xmm)} sources")

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.table import setdiff, unique

xxl_sdssqso_coords = SkyCoord(xxl_sdssqso["RAJ2000"], xxl_sdssqso["DEJ2000"])
xxl_4xmm_coords = SkyCoord(xxl_4xmm["RA_ICRS"], xxl_4xmm["DE_ICRS"])

idx, d2d, _ = xxl_4xmm_coords.match_to_catalog_sky(xxl_sdssqso_coords)

mask = d2d.arcsec < 5
qsox = xxl_sdssqso[idx[mask]]

mask = d2d.arcmin < 1
qso_near_xsources = xxl_sdssqso[idx[mask]]
qso_farfrom_xsources = setdiff(xxl_sdssqso, qso_near_xsources, keys="recno")

print(f"SDSS-QSO with 4XMM counterpart in XXL-North: {len(qsox)} sources")
print(f"SDSS-QSO far from 4XMM sources in XXL-North: {len(qso_farfrom_xsources)} sources")

Download RapidXMM data for QSO far from X-ray sources:

In [ ]:
from api import rapidxmm

qso_farfrom_xsources_coords = SkyCoord(qso_farfrom_xsources["RAJ2000"], qso_farfrom_xsources["DEJ2000"])
uls_qso_farfrom_xsources = rapidxmm.query_coords(qso_farfrom_xsources_coords, obstype="pointed", instrum="PN")

uls_qso_farfrom_xsources_file = data_folder.joinpath("uls_qso_farfrom_xsources_xxl.fits")
uls_qso_farfrom_xsources.write(uls_qso_farfrom_xsources_file, format="fits", overwrite=True)

len(uls_qso_farfrom_xsources)

Download RapidXMM data for QSO with X-ray counterparts:

In [ ]:
import numpy as np
from tqdm.auto import tqdm
from astropy.table import join, vstack


qsox_coords = SkyCoord(qsox["RAJ2000"], qsox["DEJ2000"])

ebands = ["6", "7", "8"]
uls_all = []

nsplit = np.ceil(len(qsox_coords) / 200)
for coords_segment in tqdm(np.array_split(qsox_coords, nsplit)):
    uls = rapidxmm.query_coords(coords_segment, obstype="pointed", instrum="PN")

    # Calculate count-rates
    for eband in ebands:
        uls[f"band{eband}_cr"] = (
            (uls[f"band{eband}_src_counts"] - uls[f"band{eband}_bck_counts"]) / 
            uls[f"band{eband}_exposure"] / uls["eef"]
        )
        
    uls_all.append(uls)

uls_qsox = vstack(uls_all)

len(uls_qsox)

Band 8 count-rates distirbution, including stacking results:

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['mathtext.fontset'] = "stix"
plt.rcParams['mathtext.rm'] = "STIXGeneral"
plt.rcParams['font.family'] = "STIXGeneral"
plt.rcParams['font.size'] = 13
plt.rcParams["axes.formatter.use_mathtext"] = True
plt.rcParams["legend.labelspacing"] = 0.25
    
plt.figure(figsize=(6, 2.5))

plt.subplot(121)
bins = np.logspace(-4, 1, num=21)
plt.hist(uls_qsox["band8_cr"], bins=bins, color="gray")
plt.axvline(1.5e-03, ls="--", lw=3, color="k")
plt.xlim(1e-4, 1)
plt.xscale('log')
plt.grid(ls=":", lw=1.5)
plt.xlabel("count rate (0.2-12 keV)")

plt.subplot(122)
cr_ratio = uls_qsox["band7_cr"] / uls_qsox["band6_cr"]
cr_ratio_stack = 3.4e-04 / 9.9e-04
bins = np.logspace(-3, 2, num=21)
plt.hist(cr_ratio, bins=bins, color="gray")
plt.axvline(cr_ratio_stack, ls="--", lw=3, color="k")
plt.xlim(1e-3, 100)
plt.xscale('log')
plt.grid(ls=":", lw=1.5)
plt.xlabel("hard-to-soft count rate ratio")

plt.tight_layout()
plt.savefig("stacking_qso_xxl_crdist.pdf", bbox_inches='tight', pad_inches=0)
plt.show()